In [ ]:
# default_exp pricedata

# pricedata

> pricedata is used to download/store/process equity price data

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

"""This module contains two methods of downloading price history
Yahoo Finance and Alpha Vantage are the data providers used
by faf
Copyright 2021 Fafs
License: http://creativecommons.org/licenses/by/4.0/
"""

import numpy as np
import pandas as pd
import yfinance as yf

from alpha_vantage.timeseries import TimeSeries

from typing import List


def get_history_yf(assets: List=None, 
                   start_date: str='2000-01-01',
                   end_date: str=None) -> pd.DataFrame:
    """
    Gets price history of list of assets from Yahoo Finance.
    The price data contains the following:
    Open, High, Low, Close, Adj Close, Volume

    -assets: (list of str) Asset symbols
    -start_date: (str) Starting Date
    -end_date: (str) End Date (Excluded)
    
    return: (dict of pandas.DataFrame) price history of assets
    """
    if end_date is None:
        data = yf.download(assets, start_date)
    else:
        data = yf.download(assets, start_date, end_date)
    data.rename(columns={'Adj Close': 'adj_close'}, inplace=True)
    return data


my_key = 'file' #open('../my_secrets/AlphaVantageKey').readline().strip()
ts = TimeSeries(key=my_key, output_format="pandas")

def get_history(assets: List=None, verbose: bool=True) -> pd.DataFrame:
    """
    Gets price history of list of assets from Alpha Vantage. 
    The price data contains the following:
    open, high, low, close, adj_close, volume, dividend, and split_coef

    -assets: (list of str) Asset symbols
    
    -verbose: (bool) if True print some logs
    
    return: (dict of pandas.DataFrame) price history of assets
    """
    prices = {}
    for i, asset in enumerate(assets):
        if verbose: print(asset)
        if len(asset) > 0:
            if i and i % 5 == 0:
                if verbose: print("5 assets loaded. waiting 60 seconds per AV api frequency limitations")
                sleep(60)
            data, meta_data = ts.get_daily_adjusted(asset, outputsize='full')
            if verbose: print(meta_data)
            if data.index.name.lower() == 'date':
                prices[asset] = data.rename(columns={
                    '1. open': 'open',
                    '2. high': 'high',
                    '3. low': 'low',
                    '4. close': 'close',
                    '5. adjusted close': 'adj_close',
                    '6. volume': 'volume',
                    '7. dividend amount': 'dividend',
                    '8. split coefficient': 'split_coef'}).sort_index()
            else:
                prices[asset] = data.rename(columns={
                    '1. open': 'open',
                    '2. high': 'high',
                    '3. low': 'low',
                    '4. close': 'close',
                    '5. adjusted close': 'adj_close',
                    '6. volume': 'volume',
                    '7. dividend amount': 'dividend',
                    '8. split coefficient': 'split_coef'}).sort_values(by='date', ascending=True)
    return prices

In [ ]:
show_doc(get_history_yf)

<h4 id="get_history_yf" class="doc_header"><code>get_history_yf</code><a href="__main__.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>get_history_yf</code>(**`assets`**:`List`\[`T`\]=*`None`*, **`start_date`**:`str`=*`'2000-01-01'`*, **`end_date`**:`str`=*`None`*)

Gets price history of list of assets from Yahoo Finance.
The price data contains the following:
Open, High, Low, Close, Adj Close, Volume

-assets: (list of str) Asset symbols
-start_date: (str) Starting Date
-end_date: (str) End Date (Excluded)

return: (dict of pandas.DataFrame) price history of assets

In [ ]:
show_doc(get_history)

<h4 id="get_history" class="doc_header"><code>get_history</code><a href="__main__.py#L44" class="source_link" style="float:right">[source]</a></h4>

> <code>get_history</code>(**`assets`**:`List`\[`T`\]=*`None`*, **`verbose`**:`bool`=*`True`*)

Gets price history of list of assets from Alpha Vantage. 
The price data contains the following:
open, high, low, close, adj_close, volume, dividend, and split_coef

-assets: (list of str) Asset symbols

-verbose: (bool) if True print some logs

return: (dict of pandas.DataFrame) price history of assets

In [ ]:
spy = get_history_yf('SPY', start_date='2021-06-07', end_date='2021-06-09')
spy.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,adj_close,Volume
Date,,,,,,
2021-06-07,422.589996,422.779999,421.190002,422.190002,422.190002,51555000
2021-06-08,423.109985,423.209991,420.320007,422.279999,422.279999,47134300


In [ ]:
diff = spy.loc[spy.index[-1], 'Close'] - spy.loc[spy.index[-2], 'Close']
assert np.abs(diff - 0.09) < 0.001